<a href="https://colab.research.google.com/github/MZiaAfzal71/Edge-Aware-GNN/blob/main/Models/Chemprop_D_MPNN_Repeated_KFold_with_descriptor_for_ESOL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### ⚠️ Runtime Configuration (Important)

To speed up model training, it is recommended to use a GPU runtime in Google Colab.

Go to:
- **Runtime → Change runtime type**
- Set **Hardware accelerator** to **GPU**
- Select **T4 GPU**

Using a GPU significantly reduces training time for Chemprop models.


# Chemprop on ESOL — Repeated 5×5 Cross-Validation (Structure + Descriptors)

This notebook trains and evaluates a **Chemprop message-passing neural network (MPNN)**
for predicting aqueous solubility on the **ESOL (Delaney) dataset** using a
**hybrid molecular representation**.

In this experiment:
- Molecular graphs are learned from **SMILES-derived molecular structures**
- **RDKit-computed molecular descriptors** are incorporated alongside graph representations
- Out of 217 RDKit descriptors, **198 with non-zero variance** are retained
- Descriptor features are normalized using training-fold statistics only
- Model evaluation is performed using a **5×5 repeated cross-validation** strategy
- Each fold and repeat uses identical data splits for fair comparison
- An ensemble of Chemprop models is trained for each evaluation

This notebook evaluates the impact of augmenting Chemprop’s
structure-based representations with global physicochemical descriptors
under a robust repeated cross-validation protocol.


In [2]:
# 1️⃣ Fetch data
!git clone https://github.com/MZiaAfzal71/Edge-Aware-GNN.git

Cloning into 'Combined-Interaction-Matrix'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 36 (delta 4), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (36/36), 151.67 KiB | 1.24 MiB/s, done.
Resolving deltas: 100% (4/4), done.


In [3]:
# 2️⃣ Change current/working directory
%cd Combined-Interaction-Matrix/ESOL\ Dataset

/content/Combined-Interaction-Matrix/ESOL Dataset


In [1]:
# 3️⃣ Install rdkit and chemprop
!pip install rdkit chemprop

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.1/144.1 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 846.0/846.0 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.0/188.0 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.5/849.5 kB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.0/176.0 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 59.7 MB/s eta 0:00:00


In [4]:
#  4️⃣ Imports
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm
import random
import copy
import os

import torch
from lightning import pytorch as pl
from lightning.pytorch.callbacks import ModelCheckpoint

from chemprop import data, featurizers, models, nn
from chemprop.models import save_model, load_model
from chemprop.data import split_data_by_indices, make_split_indices


from rdkit import Chem
from rdkit.Chem import Descriptors

from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import RepeatedKFold

In [5]:
# 5️⃣ Set random seeds for reproducibility across Python, NumPy, and PyTorch

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [7]:
# 6️⃣ Utility function for repeated k-fold cross-validation

def run_repeated_kfold_cv(
    df,
    n_splits=5,
    n_repeats=5,
    seed=42
):
    rkf = RepeatedKFold(
        n_splits=n_splits,
        n_repeats=n_repeats,
        random_state=seed
    )

    split_id = 0

    indices = np.arange(len(df))

    for train_idx, val_idx in rkf.split(indices):
        repeat = split_id // n_splits
        fold   = split_id % n_splits


        yield repeat, fold, train_idx, val_idx

        split_id += 1


In [8]:
# 7️⃣ Run repeated k-fold cross-validation for a Chemprop MPNN model, training on molecular graphs and reporting RMSE, R², and MAE for each repeat–fold split

def run_chemprop_repeated_fold(
    df,
    descriptors,
    smiles_col='smiles',
    target_col='measured log solubility in mols per litre',
    epochs=50
):

  fold_results = []

  for repeat, fold, train_idx, val_idx in run_repeated_kfold_cv(df):

    smis = df[smiles_col].values
    ys = df[[target_col]].values


    all_data = [data.MoleculeDatapoint.from_smi(smi, y, x_d=X_d)
            for smi, y, X_d in zip(smis, ys, descriptors)]

    train_data = [all_data[idx] for idx in train_idx]
    val_data = [all_data[idx] for idx in val_idx]

    featurizer = featurizers.SimpleMoleculeMolGraphFeaturizer()

    train_dset = data.MoleculeDataset(train_data, featurizer)
    scaler = train_dset.normalize_targets()
    extra_desc_scaler = train_dset.normalize_inputs("X_d")

    val_dset = data.MoleculeDataset(val_data, featurizer)
    val_dset.normalize_targets(scaler)
    val_dset.normalize_inputs("X_d", extra_desc_scaler)

    train_loader = data.build_dataloader(train_dset)
    train_loader_pred = data.build_dataloader(train_dset, shuffle=False)

    val_loader = data.build_dataloader(val_dset, shuffle=False)

    mp = nn.BondMessagePassing()
    agg = nn.NormAggregation()
    # agg = nn.MeanAggregation()

    ffn_input_dim = mp.output_dim  + X.shape[1]
    output_transform = nn.UnscaleTransform.from_standard_scaler(scaler)
    X_d_transform = nn.ScaleTransform.from_standard_scaler(extra_desc_scaler)
    ffn = nn.RegressionFFN(input_dim=ffn_input_dim, output_transform=output_transform)
    batch_norm = True

    metric_list = [nn.metrics.RMSE(), nn.metrics.MAE(), nn.metrics.R2Score()] # Only the first metric is used for training and early stopping

    mpnn = models.MPNN(mp, agg, ffn, X_d_transform=X_d_transform, metrics=metric_list)

    out_dir = Path(f"chemprop_model/ch_pt_repeat_{repeat}_fold_{fold}") # directory for storing the best model after training
    os.makedirs(out_dir, exist_ok=True)

    check_pointing = ModelCheckpoint(
        out_dir,  # Directory where model checkpoints will be saved
        "best-{epoch}-{val_loss:.3f}",  # Filename format for checkpoints, including epoch and validation loss
        "val_loss",  # Metric used to select the best checkpoint (based on validation loss)
        mode="min",  # Save the checkpoint with the lowest validation loss (minimization objective)
        save_last=True,  # Always save the most recent checkpoint, even if it's not the best
    )


    trainer = pl.Trainer(
        logger=False,
        enable_checkpointing=True, # Use `True` if you want to save model checkpoints. The checkpoints will be saved in the `checkpoints` folder.
        enable_progress_bar=True,
        accelerator="auto",
        devices=1,
        max_epochs=epochs, # number of epochs to train for
        callbacks=[check_pointing], # Use the configured checkpoint callback
    )
    trainer.fit(mpnn, train_loader, val_loader)

    best_model_path = check_pointing.best_model_path
    mpnn = mpnn.__class__.load_from_checkpoint(best_model_path, weights_only=False)

    train_dset = data.MoleculeDataset(train_data, featurizer)
    val_dset = data.MoleculeDataset(val_data, featurizer)

    train_loader = data.build_dataloader(train_dset, shuffle=False)
    val_loader = data.build_dataloader(val_dset, shuffle=False)

    train_predictions = torch.concat(trainer.predict(mpnn, train_loader)).cpu().numpy().ravel()
    val_predictions = torch.concat(trainer.predict(mpnn, val_loader)).cpu().numpy().ravel()

    train_y_true = df.iloc[train_idx][target_col].values.ravel()
    val_y_true = df.iloc[val_idx][target_col].values.ravel()

    train_rmse = np.sqrt(mean_squared_error(train_y_true, train_predictions))
    train_r2 = r2_score(train_y_true, train_predictions)
    print(f"\nThe training r2 score is: {train_r2}\n")

    train_mae = mean_absolute_error(train_y_true, train_predictions)

    val_rmse = np.sqrt(mean_squared_error(val_y_true, val_predictions))

    val_r2 = r2_score(val_y_true, val_predictions)
    print(f"\nThe validation r2 score is: {val_r2}\n")

    val_mae = mean_absolute_error(val_y_true, val_predictions)

    fold_results.append({
        "Repeat": repeat+1,
        "Fold" : fold+1,
        "best_train_rmse": train_rmse,
        "best_train_r2": train_r2,
        "best_train_mae": train_mae,
        "best_val_rmse": val_rmse,
        "best_val_r2": val_r2,
        "best_val_mae": val_mae
    })

  return fold_results



In [9]:
# 8️⃣ Calculates all available RDKit descriptors for a given SMILES string.

def rdkit_descriptors_from_smiles(smiles):
    """
    Parameters
    ----------
    smiles : list of smiles strings

    Returns
    -------
    pd.DataFrame
        DataFrame with descriptor names as columns.
        Returns NaN values if SMILES is invalid.
    """
    # Get descriptor names and functions
    rdkit_descs = []

    desc_list = Descriptors.descList
    desc_names = [name for name, _ in desc_list]

    # Initialize output with NaNs
    # values = [np.nan] * len(desc_names)

    for sm in tqdm(smiles, total=len(smiles)):
      sm_descs = []
      mol = Chem.MolFromSmiles(sm)
      if mol is None:
          rdkit_descs.append([]*len(desc_names))
          continue

      for _, func in desc_list:
          try:
              sm_descs.append(func(mol))
          except Exception:
              sm_descs.append(np.nan)

      rdkit_descs.append(sm_descs)

    return pd.DataFrame(rdkit_descs, columns=desc_names)

In [10]:
# 9️⃣ Load dataset, standardize target variable, and prepare data for modeling

tqdm.pandas()

file_path = "delaney-processed-scaffold.csv"
smiles_col = "smiles"
target_col = "measured log solubility in mols per litre"

set_seed(42)

df = pd.read_csv(file_path)

rdkit_descriptors = rdkit_descriptors_from_smiles(df[smiles_col])
variance_df = rdkit_descriptors.var()
zero_var_columns = variance_df[variance_df == 0].index.tolist()
cleaned_rdkit_descs = rdkit_descriptors.drop(columns = zero_var_columns)
X = cleaned_rdkit_descs.values

  0%|          | 0/1128 [00:00<?, ?it/s]

In [11]:
# 🔟 Execute repeated k-fold Chemprop training, collect fold-wise performance metrics, and save the results to a CSV file

results = run_chemprop_repeated_fold(df, X, epochs=50)
res_df = pd.DataFrame(results)
res_df.to_csv("chemprop repeated kfold with descriptors.csv", index=False)

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: Loading `train_dataloader` to estimate number of stepping batches.
INFO:lightning.pytorch.utilities.rank_zero:Loading `train_dataloader` to estimate number of stepping batches.


┏━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name            ┃ Type               ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ message_passing │ BondMessagePassing │  227 K │ train │     0 │
│ 1 │ agg             │ NormAggregation    │      0 │ train │     0 │
│ 2 │ bn              │ Identity           │      0 │ train │     0 │
│ 3 │ predictor       │ RegressionFFN      │  150 K │ train │     0 │
│ 4 │ X_d_transform   │ ScaleTransform     │      0 │ train │     0 │
│ 5 │ metrics         │ ModuleList         │      0 │ train │     0 │
└───┴─────────────────┴────────────────────┴────────┴───────┴───────┘

Trainable params: 377 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 377 K                                                                                                
Total estimated model params size (MB): 1                                                                          
Modules in train mode: 26                                                                                          
Modules in eval mode: 0                                                                                            
Total FLOPs: 0

Output()

INFO: `Trainer.fit` stopped: `max_epochs=50` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=50` reached.


...

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Output()

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Output()


The training r2 score is: 0.9791693573945894


The validation r2 score is: 0.9312947590881893



In [ ]:
# 1️⃣1️⃣

In [ ]:
# 1️⃣2️⃣

In [ ]:
# 1️⃣3️⃣

In [ ]:
# 1️⃣4️⃣

In [ ]:
# 1️⃣5️⃣

In [ ]:
# 1️⃣6️⃣

In [ ]:
# 1️⃣7️⃣

In [ ]:
# 1️⃣8️⃣

In [ ]:
# 1️⃣9️⃣